# Models generation
This script builds the network (CNN) and generates 9 models using different folds

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

import cv2
import os

## GPU is not necessary but advised!
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

2024-01-11 11:42:05.261225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-11 11:42:05.322551: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-11 11:42:05.657718: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/paulo/miniconda3/envs/tf/lib/
2024-01-11 11:42:05.657765: W tensorflow/compiler/

Default GPU Device: /device:GPU:0


2024-01-11 11:42:06.082979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-11 11:42:06.143505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-11 11:42:06.157846: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-11 11:42:06.158079: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there

## Load images

In [9]:
## 18 nondyslectic and 18 dyslectic subjects
non_parts = [ 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,15,16,18,19,20,21,22]
dys_parts = [23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,39,40,41]

##
# Loading GSSP images and dividing them to training and testing sets using 'testIndexes'
def load_img(indir,testIndexes):
    trainSamples = []
    trainLabels = []
    testSamples = []
    testLabels = []
    for file in os.listdir(indir):
        #print(file)
        if not file.endswith('jpg'):
            continue
        part = int(file[1:3])
        if part not in non_parts and part not in dys_parts:
            continue
        image = cv2.imread("{}/{}".format(indir,file))
        ## TAKE ONLY FIRST 150 pixels (equivalent of first 15 seconds of the recording)
        image = image[:150,:150,:]
        if not part in testIndexes:
            trainSamples.append(image)
            if part<=22:
                trainLabels.append(0)
            else:
                trainLabels.append(1)
        else: #test
            testSamples.append(image)
            if part<=22:
                testLabels.append(0)
            else:
                testLabels.append(1)
    return np.array(trainSamples),np.array(trainLabels),np.array(testSamples),np.array(testLabels)

In [12]:
##
# Folds creation

folds = []
for i in range(0,18,2):
    folds.append([non_parts[i],dys_parts[i],non_parts[i+1],dys_parts[i+1]])

print(folds)

[[2, 23, 3, 24], [4, 25, 5, 26], [6, 27, 7, 28], [8, 29, 9, 30], [10, 31, 11, 33], [12, 34, 15, 35], [16, 36, 18, 37], [19, 38, 20, 39], [21, 40, 22, 41]]


In [ ]:
##
# Create the network architecture

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
def create_model():
    model = Sequential()
    filter_size = (5,5)
    model.add(Conv2D(16, filter_size, padding="same",input_shape=(150,150,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, filter_size, padding="same",activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, filter_size, padding="same",activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, filter_size, padding="same",activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512,activation='sigmoid'))
    model.add(Dense(256,activation='sigmoid'))
    model.add(Dense(128,activation='sigmoid'))
    model.add(Dense(1,activation='sigmoid'))
    return model
model = create_model()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

In [13]:
##
# Building 9 models - one for each set of 8 folds

from collections import Counter
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
for i,f in enumerate(folds):
    print('='*100)
    print("FOLD",i,"   ",f)
    print('='*100)
    trainSamples, trainLabels, testSamples, testLabels = load_img('gssp',f)
    print('loaded train samples ',len(trainSamples))
    print('loaded test samples ',len(testSamples))

    print("Train labels distribution:", Counter(trainLabels))
    print("Test labels distribution:", Counter(testLabels))
    lr = 0.001
    model = create_model()
    opt = tf.keras.optimizers.SGD(learning_rate=lr)
    model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])

    #checkpt = ModelCheckpoint (filepath=f'f10_{i}_model.h5', save_best_only=True)
    #estopping = EarlyStopping(monitor='loss', patience=50, verbose=1)
    
    EPOCHS = 200
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS, batch_size=32, validation_data=(testSamples,testLabels),
            #     callbacks = [estopping,checkpt]
            ) 
    model.save(f"final_f{i}_model.h5")

[[2, 23, 3, 24], [4, 25, 5, 26], [6, 27, 7, 28], [8, 29, 9, 30], [10, 31, 11, 33], [12, 34, 15, 35], [16, 36, 18, 37], [19, 38, 20, 39], [21, 40, 22, 41]]
FOLD 0     [2, 23, 3, 24]
loaded train samples  3138
loaded test samples  183
Train labels distribution: Counter({1: 2175, 0: 963})
Test labels distribution: Counter({0: 123, 1: 60})
Epoch 1/200
 1/99 [..............................] - ETA: 47s - loss: 0.6003 - accuracy: 0.7812

2024-01-11 16:59:59.581894: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_11/dropout_11/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


99/99 [==============================] - 2s 20ms/step - loss: 0.6205 - accuracy: 0.6931 - val_loss: 0.8562 - val_accuracy: 0.3279
Epoch 2/200
99/99 [==============================] - 2s 16ms/step - loss: 0.6096 - accuracy: 0.6931 - val_loss: 0.8748 - val_accuracy: 0.3279
Epoch 3/200
99/99 [==============================] - 2s 15ms/step - loss: 0.6053 - accuracy: 0.6931 - val_loss: 0.8971 - val_accuracy: 0.3279
Epoch 4/200
99/99 [==============================] - 2s 15ms/step - loss: 0.6023 - accuracy: 0.6931 - val_loss: 0.8946 - val_accuracy: 0.3279
Epoch 5/200
99/99 [==============================] - 2s 16ms/step - loss: 0.5998 - accuracy: 0.6931 - val_loss: 0.8889 - val_accuracy: 0.3279
Epoch 6/200
99/99 [==============================] - 2s 16ms/step - loss: 0.5974 - accuracy: 0.6931 - val_loss: 0.8675 - val_accuracy: 0.3279
Epoch 7/200
99/99 [==============================] - 2s 15ms/step - loss: 0.5952 - accuracy: 0.6931 - val_loss: 0.8655 - val_accuracy: 0.3279
Epoch 8/200
99/99 

2024-01-11 17:05:19.175337: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_12/dropout_12/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


98/98 [==============================] - 2s 20ms/step - loss: 0.8112 - accuracy: 0.4113 - val_loss: 0.7091 - val_accuracy: 0.4929
Epoch 2/200
98/98 [==============================] - 2s 16ms/step - loss: 0.6362 - accuracy: 0.6852 - val_loss: 0.7481 - val_accuracy: 0.4929
Epoch 3/200
98/98 [==============================] - 2s 16ms/step - loss: 0.6229 - accuracy: 0.6852 - val_loss: 0.7589 - val_accuracy: 0.4929
Epoch 4/200
98/98 [==============================] - 2s 15ms/step - loss: 0.6194 - accuracy: 0.6852 - val_loss: 0.7549 - val_accuracy: 0.4929
Epoch 5/200
98/98 [==============================] - 2s 16ms/step - loss: 0.6168 - accuracy: 0.6852 - val_loss: 0.7556 - val_accuracy: 0.4929
Epoch 6/200
98/98 [==============================] - 2s 17ms/step - loss: 0.6144 - accuracy: 0.6852 - val_loss: 0.7455 - val_accuracy: 0.4929
Epoch 7/200
98/98 [==============================] - 2s 16ms/step - loss: 0.6120 - accuracy: 0.6852 - val_loss: 0.7444 - val_accuracy: 0.4929
Epoch 8/200
98/98 

...

99/99 [==============================] - 2s 19ms/step - loss: 0.7744 - accuracy: 0.4395 - val_loss: 0.7245 - val_accuracy: 0.4294
Epoch 2/200
99/99 [==============================] - 2s 16ms/step - loss: 0.6328 - accuracy: 0.6856 - val_loss: 0.7807 - val_accuracy: 0.4294
Epoch 3/200
99/99 [==============================] - 2s 16ms/step - loss: 0.6206 - accuracy: 0.6856 - val_loss: 0.7995 - val_accuracy: 0.4294
Epoch 4/200
99/99 [==============================] - 2s 16ms/step - loss: 0.6168 - accuracy: 0.6856 - val_loss: 0.7994 - val_accuracy: 0.4294
Epoch 5/200
99/99 [==============================] - 2s 16ms/step - loss: 0.6138 - accuracy: 0.6856 - val_loss: 0.7962 - val_accuracy: 0.4294
Epoch 6/200
99/99 [==============================] - 2s 16ms/step - loss: 0.6112 - accuracy: 0.6856 - val_loss: 0.7947 - val_accuracy: 0.4294
Epoch 7/200
99/99 [==============================] - 2s 16ms/step - loss: 0.6090 - accuracy: 0.6856 - val_loss: 0.7927 - val_accuracy: 0.4294
Epoch 8/200
99/99 

2024-01-11 17:37:29.496919: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_18/dropout_18/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


100/100 [==============================] - 2s 20ms/step - loss: 0.6316 - accuracy: 0.6812 - val_loss: 0.7468 - val_accuracy: 0.4895
Epoch 2/200
100/100 [==============================] - 2s 15ms/step - loss: 0.6234 - accuracy: 0.6812 - val_loss: 0.7511 - val_accuracy: 0.4895
Epoch 3/200
100/100 [==============================] - 2s 16ms/step - loss: 0.6196 - accuracy: 0.6812 - val_loss: 0.7463 - val_accuracy: 0.4895
Epoch 4/200
100/100 [==============================] - 2s 16ms/step - loss: 0.6164 - accuracy: 0.6812 - val_loss: 0.7465 - val_accuracy: 0.4895
Epoch 5/200
100/100 [==============================] - 2s 16ms/step - loss: 0.6141 - accuracy: 0.6812 - val_loss: 0.7421 - val_accuracy: 0.4895
Epoch 6/200
100/100 [==============================] - 2s 16ms/step - loss: 0.6114 - accuracy: 0.6812 - val_loss: 0.7308 - val_accuracy: 0.4895
Epoch 7/200
100/100 [==============================] - 2s 17ms/step - loss: 0.6091 - accuracy: 0.6812 - val_loss: 0.7217 - val_accuracy: 0.4895
Epoc

2024-01-11 17:42:56.374744: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_19/dropout_19/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


97/97 [==============================] - 2s 17ms/step - loss: 0.6405 - accuracy: 0.6773 - val_loss: 0.6612 - val_accuracy: 0.6194
Epoch 2/200
97/97 [==============================] - 2s 16ms/step - loss: 0.6265 - accuracy: 0.6773 - val_loss: 0.6594 - val_accuracy: 0.6194
Epoch 3/200
97/97 [==============================] - 2s 16ms/step - loss: 0.6223 - accuracy: 0.6773 - val_loss: 0.6570 - val_accuracy: 0.6194
Epoch 4/200
97/97 [==============================] - 2s 17ms/step - loss: 0.6193 - accuracy: 0.6773 - val_loss: 0.6524 - val_accuracy: 0.6194
Epoch 5/200
97/97 [==============================] - 2s 17ms/step - loss: 0.6164 - accuracy: 0.6773 - val_loss: 0.6456 - val_accuracy: 0.6194
Epoch 6/200
97/97 [==============================] - 2s 16ms/step - loss: 0.6143 - accuracy: 0.6773 - val_loss: 0.6428 - val_accuracy: 0.6194
Epoch 7/200
97/97 [==============================] - 2s 18ms/step - loss: 0.6121 - accuracy: 0.6773 - val_loss: 0.6377 - val_accuracy: 0.6194
Epoch 8/200
97/97 